In [3]:
import requests
import json
import folium

def get_tube_incidents():
    url = "https://api.tfl.gov.uk/Line/Mode/tube/Status"
    response = requests.get(url)

    if response.status_code == 200:
        data = json.loads(response.text)
        incidents = []
        for item in data:
            if "lineStatuses" in item and len(item["lineStatuses"]) > 0:
                if "statusSeverityDescription" in item["lineStatuses"][0]:
                    line_name = item["name"]
                    line_status = item["lineStatuses"][0]["statusSeverityDescription"]
                    incidents.append({"line_name": line_name, "line_status": line_status})
        return incidents
    else:
        return None

if __name__ == "__main__":
    incidents = get_tube_incidents()
    if incidents:
        london_map = folium.Map(location=[51.5074, -0.1278], zoom_start=11)
        for incident in incidents:
            if incident["line_status"] == "Good Service":
                color = "green"
            else:
                color = "red"
            folium.Marker(
                location=[51.5074, -0.1278],
                popup=f"{incident['line_name']}: {incident['line_status']}",
                icon=folium.Icon(color=color)
            ).add_to(london_map)
        london_map.save("london_tube_map.html")
    else:
        print("Could not retrieve live updates of incidents on the London tube network.")
